# Importing Dependencies

In [2]:
import os
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# Load Environment

In [11]:
env_name = "CartPole-v1"
env = gym.make(env_name, render_mode="human")

In [12]:
episodes = 5

for episode in range(1, episodes + 1):
    state, info = env.reset()
    done = False
    score = 0

    while not done:
        action = env.action_space.sample()
        next_state, reward, terminated, truncated, info = env.step(action)

        done = terminated or truncated
        score += reward
        state = next_state

    print(f"Episode {episode} score: {score}")

env.close()

Episode 1 score: 42.0
Episode 2 score: 15.0
Episode 3 score: 42.0
Episode 4 score: 13.0
Episode 5 score: 16.0


## Understanding the Environment

In [17]:
env.action_space

Discrete(2)

In [18]:
env.observation_space

Box([-4.8               -inf -0.41887903        -inf], [4.8               inf 0.41887903        inf], (4,), float32)

In [19]:
env.action_space.sample()

np.int64(1)

In [20]:
env.observation_space.sample()

array([-0.26743934, -0.3295444 , -0.19192973,  0.5796551 ], dtype=float32)

# Train RL Model

In [23]:
log_path = os.path.join("training", "logs")
os.makedirs(log_path, exist_ok=True)

In [25]:
env = gym.make(env_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [27]:
model.learn(total_timesteps=100000)

Logging to training\logs\PPO_1
-----------------------------
| time/              |      |
|    fps             | 1565 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1046        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008467808 |
|    clip_fraction        | 0.0859      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | 0.000752    |
|    learning_rate        | 0.0003      |
|    loss                 | 7.87        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0131     |
|    value_loss           | 54.7        |
-----------------------------------------
---

# Save and Reload Model

In [28]:
PPO_Path = os.path.join("training", "saved models", "PPO_Model_CartPole")
model.save(PPO_Path)

D:\My_Projects\reinforcement-learning-lab\.venv\Lib\site-packages\stable_baselines3\common\save_util.py:284: UserWarning: Path 'training\saved models' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [30]:
del model
model = PPO.load(PPO_Path, env=env)

# Evaluation

In [32]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(np.float64(500.0), np.float64(0.0))

# Test Model

In [35]:
episodes = 5

for episode in range(1, episodes + 1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, dones, infos = env.step(action)

        done = dones[0]
        score += reward[0]

    print(f"Episode {episode} score: {score}")

env.close()

Episode 1 score: 500.0
Episode 2 score: 500.0
Episode 3 score: 500.0
Episode 4 score: 500.0
Episode 5 score: 500.0


In [37]:
obs = env.reset()
action, _ = model.predict(obs)
env.step(action)

(array([[ 0.0399088 ,  0.2107481 ,  0.01285774, -0.27863166]],
       dtype=float32),
 array([1.], dtype=float32),
 array([False]),
 [{'TimeLimit.truncated': False}])

# View Logs in Tensorboard

In [39]:
training_logs_path = os.path.join(log_path, "PPO_1")

In [ ]:
!tensorboard --logdir={training_logs_path}

1. Open your command prompt or terminal.
2. Run the following command:
   tensorboard --logdir=training_logs_path

   (Replace `training_logs_path` with the path to your training logs directory.)

3. After TensorBoard starts, open a web browser and go to:
   http://localhost:6006/

# Adding a callback to the training Stage

In [40]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [42]:
save_path = os.path.join("training", "saved models")
os.makedirs(save_path, exist_ok=True)

In [43]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)
eval_callback = EvalCallback(env,
                             callback_on_new_best=stop_callback,
                             eval_freq=10000,
                             best_model_save_path=save_path,
                             verbose=1)

In [44]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [45]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to training\logs\PPO_2
-----------------------------
| time/              |      |
|    fps             | 1006 |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 875         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009713139 |
|    clip_fraction        | 0.0911      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.687      |
|    explained_variance   | -0.0171     |
|    learning_rate        | 0.0003      |
|    loss                 | 5.3         |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0127     |
|    value_loss           | 42          |
-----------------------------------------
---

# Changing Policies

In [50]:
net_arch = dict(
    pi=[128, 128, 128, 128],
    vf=[128, 128, 128, 128]
)

In [51]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path, policy_kwargs={'net_arch': net_arch})

Using cpu device


In [52]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to training\logs\PPO_3
-----------------------------
| time/              |      |
|    fps             | 1139 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 817         |
|    iterations           | 2           |
|    time_elapsed         | 5           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.014326224 |
|    clip_fraction        | 0.194       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.682      |
|    explained_variance   | -0.00161    |
|    learning_rate        | 0.0003      |
|    loss                 | 3.3         |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0239     |
|    value_loss           | 20.8        |
-----------------------------------------
---

# Using an Alternate Algorithm

In [54]:
from stable_baselines3 import DQN

In [55]:
model = DQN('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [56]:
model.learn(total_timesteps=20000)

Logging to training\logs\DQN_1
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.964    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 3087     |
|    time_elapsed     | 0        |
|    total_timesteps  | 76       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.907    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 866      |
|    time_elapsed     | 0        |
|    total_timesteps  | 196      |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.484    |
|    n_updates        | 23       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.878    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 